In [1]:
import pandas as pd
# import seaborn as sns
import numpy as np
# import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize

In [11]:
data_path = '../data/'
rdfTrain = pd.read_csv(data_path + 'train.tsv', sep = '\t', header = None, names = ['text', 'label', 'id'])
rdfTrain.head()

,text,label,id
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj


## Neutral/Non-neutral

In [19]:
def label_neutral(row):
    if row['label'] == '27':
        return 1
    else:
        return 0

In [14]:
dfTrain = rdfTrain
dfTrain["neutral"] = dfTrain.apply(lambda row: label_neutral(row), axis = 1)
dfTrain

,text,label,id,neutral
0,My favourite food is anything I didn't have to...,27,eebbqej,1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,0
3,To make her feel threatened,14,ed7ypvh,0
4,Dirty Southern Wankers,3,ed0bdzj,0
...,...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,18,edsb738,0
43406,Always thought that was funny but is it a refe...,6,ee7fdou,0
43407,What are you talking about? Anything bad that ...,3,efgbhks,0
43408,"More like a baptism, with sexy results!",13,ed1naf8,0


In [15]:
dfTrain.neutral.value_counts()

0    30587
1    12823
Name: neutral, dtype: int64

In [18]:
# validation set
data_path = '../data/'
rdfDev = pd.read_csv(data_path + 'dev.tsv', sep = '\t', header = None, names = ['text', 'label', 'id'])
rdfDev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5426 entries, 0 to 5425
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5426 non-null   object
 1   label   5426 non-null   object
 2   id      5426 non-null   object
dtypes: object(3)
memory usage: 127.3+ KB


In [22]:
dfDev = rdfDev
dfDev["neutral"] = dfDev.apply(lambda row: label_neutral(row), axis = 1)
dfDev.neutral.value_counts()

0    3834
1    1592
Name: neutral, dtype: int64

###  1. Bag of Words

In [17]:
xTrain, yTrain = dfTrain['text'], dfTrain['neutral']

In [23]:
xDev, yDev = dfDev['text'], dfDev['neutral']

#### Feature Generation

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True, stop_words='english', ngram_range = (1,1), tokenizer = token.tokenize)
text_counts = cv.fit_transform(dfTrain['text'])

#### Splitting data

In [27]:
xTrain, yTrain = text_counts, dfTrain['neutral']

In [33]:
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True, stop_words='english', ngram_range = (1,1), tokenizer = token.tokenize)
text_counts_dev = cv.fit_transform(dfDev['text'])

In [34]:
xDev, yDev = text_counts_dev, dfDev['neutral']

#### Modelling - Naive Bayes

In [36]:
from sklearn.naive_bayes import MultinomialNB
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(xTrain, yTrain)
predicted= clf.predict(xDev)
print("MultinomialNB Accuracy:",metrics.accuracy_score(yDev, predicted))

ValueError: dimension mismatch